In [1]:
import numpy as np
import tensorflow as tf
import pandas as pd
from IPython.display import display
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras import activations
import json
from tokenizer.tokenizer import create_numerate_array
from keras import backend as K
import matplotlib.pyplot as plt

In [2]:
trailers_csv = "/Volumes/Seagate/natasha-diploma/trailers.csv"
trailers_df = pd.read_csv(trailers_csv, index_col=None, header=0)
trailers_df.drop(trailers_df.columns[trailers_df.columns.str.contains('Unnamed',case = False)],axis = 1, inplace = True)
display(trailers_df.head())

,duration,release_date,restrict,orig_title,kinopoisk_id,imdb_rating,kinopoisk_rating,synopsis,categories,genres,...,emotions,Place of act,Qualities,Time of act,Based on,Audience,Mood,Subgenre,About,Theme
0,5816.0,2019-04-05,18.0,Phil,960568,5.4,5.86,"Фил – стоматолог, страдающий от депрессии. Его...",{Фильмы},"{Драмы,Зарубежные,Комедии}",...,"{'neutral': 0.5312193632125854, 'negative': 0....",['США'],['дебютный'],"['21-й век', 'новейшее время']",['оригинального сценария'],"['для молодёжи', 'для взрослых']","['грустный', 'депрессивный', 'трогательный']","['Драмы', 'Комедийные']","['преследования', 'самозванцы', 'расследование...","['развлекательные', 'психологический']"
1,5123.0,2019-04-02,18.0,Tooth Fairy,1126022,2.2,0.00,Старая женщина начинает рассказывать легенду о...,{Фильмы},"{Зарубежные,Триллеры,Ужасы}",...,"{'negative': 0.4688006341457367, 'skip': 0.392...",['не определено'],"['зрелищные', 'жанровый', 'неожиданный финал']",['не определено'],"['эпоса и легенд', 'оригинального сценария']","['для взрослых', 'для мужчин', 'для подготовле...","['трагичные', 'мрачные', 'шокирующие', 'захват...","['ужасы', 'триллер']","['преследования', 'страхи', 'дети', 'молодежь'...",['развлекательные']
2,4667.0,2019-01-19,16.0,Wake Up,1069713,2.9,0.00,"Врач-психиатр исследует дневник девушки, котор...",{Фильмы},"{Боевики,Зарубежные,Криминал,Триллеры,Ужасы}",...,"{'positive': 0.3140605390071869, 'negative': 0...",['не определено'],"['жанровый', 'дебютный']",['новейшее время'],['оригинального сценария'],"['для взрослых', 'для подготовленного зрителя'...","['захватывающие', 'жестокие', 'напряженные', '...","['Драмы', 'Боевики', 'триллер']","['психотерапевты', 'кровь', 'тайна смерти', 'п...","['психологический', 'развлекательные']"
3,6148.0,2019-08-09,16.0,The Kitchen,1117988,5.5,5.50,"Смотрите в 4К! Три жены ирландских гангстеров,...",{Фильмы},"{Боевики,Драмы,Зарубежные,Криминал,По комиксам}",...,"{'neutral': 0.4765896201133728, 'negative': 0....","['большой город', 'США']","['жанровый', 'остросюжетные']",['20-й век'],['комиксов'],"['для мужчин', 'для женщин', 'для взрослых']","['захватывающие', 'авантюрный', 'жестокие', 'н...","['Боевики', 'Драмы', 'криминальные']","['враги или вражда', 'женщины', 'антигерои', '...","['авантюрные', 'гангстерская']"
4,6013.0,2019-05-02,12.0,Pokémon Detective Pikachu,994864,6.6,6.56,Смотрите в 4K! Элитный детектив Пикачу отправл...,"{Для детей,Фильмы}","{Детективы,Зарубежные,Комедии,Семейные,Фильмы,...",...,"{'neutral': 0.4532718360424042, 'negative': 0....","['вымышленная страна (город)', 'большой город']","['частично анимационный', 'зрелищные', 'кассов...",['21-й век'],"['оригинального сценария', 'сериалов']","['для детей 13-16 лет', '9 лет', '8 лет', '7-1...","['авантюрный', 'захватывающие', 'динамичный', ...","['детские', 'Приключенческие', 'Фантастические...","['притяжение противоположностей', 'риск', 'пре...","['развлекательные', 'образовательный']"


In [3]:
sessions_csv = "/Volumes/Seagate/natasha-diploma/sessions1.csv"
sessions_df = pd.read_csv(sessions_csv, index_col=None, header=0)
sessions_df.drop(sessions_df.columns[sessions_df.columns.str.contains('Unnamed',case = False)],axis = 1, inplace = True)
display(sessions_df)

,0,1,2,3,4,5,6,7,8,9
0,403a42b604f504069c41d39c5a4fc2,0dbe493275a25285829b979316d566,9aecf32a60e4e0f7f967caecb6c798,23ee5640ec588ffaa78afa3268b04a,5b150a1cae009459493bd0fe5c7b3e,e291f8ae1de4a38c71027e9c91d51c,34cfd44ad5cc2d63441ebf437eccb5,26b3f811a6b9ed75d3cc5198105fa4,06132809afbe1c781e1b9605452081,67354be5f6f02a74e7661f9529daeb
1,d66440e7524c0a8b9b206fbb7295ba,a6d83ff19ed58d43644ca319a2c5d4,a98af02d324af4f43b7a42c82b73d0,62e8a540d2321f61f2b47dd26103e4,807d308f6b0d8df70b5ffbd5b88f38,7f778c07837b7646904a091a481145,76dc726c7fb950a0fff79073b229f6,930e6d99ffcb060d6b030d4cc90680,4b7669ea29a6770d7ee65e57b34e63,44befb98302cfa436627fa3af38642
2,231b80c884b2f1390e17f09a187889,bd6707bb04cfb8880a5f783c85be20,d6159553e41857fe23df9d3e746284,c0314a446fbef65ce9626b037a70da,961c336b0c2fa04c883c639dfee1ba,514f3ee1f00df05dc3900251972245,2c72ca8794dd4eb8f5ae90ca0dce24,c292588c17f2fb55db17a98774ef07,40fa578cd222749786f5aa816ad3c5,7e2582b83c27340f388fdce46a9209
3,d50faca942c93b02863dc4cc86a839,2530d7a6ecf471f695a2ef7a8a4191,f3153c49fddebe52085185876b61c5,914880359e59dbe7b7e60c4ddc82f4,4247088ee2239f9885a836fe4735cb,9914764b34cff2aac25a82a33d5494,6fcff414343456465369b05f6851eb,41b772a5b18bc724562bfb60f4f3ac,34cfd44ad5cc2d63441ebf437eccb5,e291f8ae1de4a38c71027e9c91d51c
4,5b150a1cae009459493bd0fe5c7b3e,e291f8ae1de4a38c71027e9c91d51c,34cfd44ad5cc2d63441ebf437eccb5,23ee5640ec588ffaa78afa3268b04a,06132809afbe1c781e1b9605452081,67354be5f6f02a74e7661f9529daeb,9aecf32a60e4e0f7f967caecb6c798,403a42b604f504069c41d39c5a4fc2,209f1d1349a473f3eb0b285140c70a,c775ad98eb2cb98e5dd370867bd0cf
...,...,...,...,...,...,...,...,...,...,...
368,a0b55fcae980de79f193f65ca74e80,417164f560d229056b4a7570815c48,568585bc69d537f6e8c68542f84ded,4d90b6cad5f3d54df5e10ec848d47b,a6caf80f7e1afa96600ae6887a56a2,86590a559c1fbf5a853b0250d2e90d,ccffca565b60e96ba17c4e8dc41781,cf9011da196f6aaa5ab407d4aed89f,01c8ee28cc66e7d7801786ba6c8d5f,199cf6d89a2f06fc552ebf86105d7d
369,57a6c27ac59feef428160b724002b5,c582be331080e8790d2899911f3242,d5d0865f3d923e6d753ceae84480b7,c045708b9c384920790737be68d28a,5b09c22ef1a754afad3a1062f4a78e,44b66885feb100106b907262835a46,45be99f14b6b78a5fceb2a7489e8b8,58a923b314735ae6f938d4c51c85f6,637463972c3f9869044cdffaf98c8d,fcdebb544d991f71e3abe0c5b09af2
370,5d9b657b32bff89dedf7c9480ba3e6,41b772a5b18bc724562bfb60f4f3ac,914880359e59dbe7b7e60c4ddc82f4,64679d8550ae26a61ee8ea94ac3fa0,26b3f811a6b9ed75d3cc5198105fa4,e3e5642e76f638dd918e718dfefe3a,ea4f9a22de6d23ff14b645e975230e,87de245b87c3c2ec94b13cef2ae2d1,23ee5640ec588ffaa78afa3268b04a,5b150a1cae009459493bd0fe5c7b3e
371,127c45352d50fad1cd1936b2e21354,961c336b0c2fa04c883c639dfee1ba,514f3ee1f00df05dc3900251972245,2c72ca8794dd4eb8f5ae90ca0dce24,c292588c17f2fb55db17a98774ef07,40b89ab9386c18e49053bf2a65bca8,231b80c884b2f1390e17f09a187889,d6159553e41857fe23df9d3e746284,c0314a446fbef65ce9626b037a70da,bd6707bb04cfb8880a5f783c85be20


In [4]:
movies_id = sessions_df.to_numpy()
copy = movies_id
np.random.shuffle(copy)
movies_id = np.concatenate((np.array(movies_id).flatten(), copy.flatten()))
                           
vocab = np.unique(movies_id)
vocab_len = len(vocab)

print(vocab_len)
print(movies_id.shape)

458
(7460,)


In [5]:
def toNumber(_str, bias):
    return len(_str) + bias

In [6]:
def getVideoInfo(info_df, trailers_row, mean=True):
    tokenized_hue = create_numerate_array(info_df['hue'].to_numpy())
    counts = np.argmax(np.bincount(tokenized_hue))
    
    key = trailers_row['Place of act'].keys()[0]
    
    scenes_array = [
        info_df['brightness'].to_numpy(),
        info_df['colorfulness'].to_numpy(),
        info_df['energy'].to_numpy(),
        info_df['tempo'].to_numpy(),
        info_df['amplitude'].to_numpy(),
        info_df['mfcc'].to_numpy(),
    ]
    scenes_array = np.nan_to_num(scenes_array, copy=False, nan=0.0, posinf=0.0, neginf=0.0)
    scaler = MinMaxScaler((-1, 1))
    scenes_array = scaler.fit_transform(scenes_array)
    
    if mean:
        new_scenes_array = []
        for i in range(len(scenes_array)):
            new_scenes_array.append(np.around(np.mean(scenes_array[i]), 3))
            
        scenes_array = new_scenes_array
    
    scenes_array.append(np.nan_to_num([trailers_row['imdb_rating'][key]], copy=False, nan=0.0, posinf=0.0, neginf=0.0)[0])
    emotions = json.loads(trailers_row['emotions'][key].replace("\'", "\""))
    scenes_array.append(np.around(emotions["negative"], 3))
    scenes_array.append(np.around(emotions["neutral"], 3))
    scenes_array.append(np.around(emotions["positive"], 3))
        
    scenes_array.append(counts)
    
    print(scenes_array)
    
    return scenes_array

In [7]:
info = {}
scenes_array = []
info_folder = "/Volumes/Seagate/natasha-diploma/videoinfo"
for i in range(0, len(movies_id)):
    info_csv = info_folder + '/' + movies_id[i] + '.csv'
    info_df =  pd.read_csv(info_csv, index_col=None, header=0)
    info[movies_id[i]] = getVideoInfo(info_df.iloc[: , 4:], trailers_df.loc[trailers_df['trailers_name'] == movies_id[i]])
    scenes_array.append(info[movies_id[i]])


[-0.211, 0.022, -0.58, 0.987, -1.0, -0.693, 5.2, 0.207, 0.484, 0.028, 5]
[-0.015, -0.03, -0.542, 0.994, -1.0, -0.67, 6.2, 0.169, 0.461, 0.068, 6]
[-0.105, -0.104, -0.62, 1.0, -1.0, -0.722, 6.0, 0.217, 0.446, 0.068, 5]
[-0.157, -0.29, -0.718, 0.989, -1.0, -0.791, 5.8, 0.173, 0.43, 0.066, 5]
[-0.34, -0.097, -0.594, 0.969, -1.0, -0.705, 5.0, 0.156, 0.423, 0.06, 4]
[-0.333, -0.279, -0.555, 1.0, -1.0, -0.718, 6.1, 0.207, 0.415, 0.046, 4]
[-0.316, -0.348, -0.675, 1.0, -1.0, -0.814, 6.8, 0.202, 0.385, 0.09, 3]
[-0.242, -0.061, -0.614, 1.0, -1.0, -0.7, 0.0, 0.228, 0.438, 0.052, 5]
[-0.062, -0.076, -0.62, 0.986, -1.0, -0.774, 6.3, 0.187, 0.423, 0.08, 3]
[-0.257, -0.302, -0.69, 1.0, -1.0, -0.748, 5.9, 0.16, 0.453, 0.033, 2]
[-0.384, -0.241, -0.677, 1.0, -1.0, -0.818, 0.0, 0.173, 0.593, 0.098, 1]
[-0.287, -0.086, -0.604, 1.0, -1.0, -0.737, 5.4, 0.144, 0.608, 0.09, 5]
[-0.337, -0.189, -0.605, 0.998, -1.0, -0.689, 0.0, 0.152, 0.615, 0.07, 5]
[-0.358, -0.262, -0.666, 1.0, -1.0, -0.72, 5.6, 0.187, 0.

[-0.259, -0.097, -0.607, 0.998, -1.0, -0.732, 7.4, 0.269, 0.363, 0.14, 5]
[-0.321, -0.188, -0.644, 1.0, -1.0, -0.688, 4.6, 0.269, 0.349, 0.101, 4]
[0.048, -0.253, -0.596, 1.0, -1.0, -0.766, 4.3, 0.234, 0.342, 0.078, 4]
[-0.256, -0.24, -0.734, 1.0, -1.0, -0.865, 6.6, 0.212, 0.335, 0.07, 1]
[-0.108, -0.089, -0.696, 0.976, -1.0, -0.778, 5.2, 0.239, 0.307, 0.052, 5]
[-0.076, -0.241, -0.697, 1.0, -1.0, -0.781, 0.0, 0.197, 0.288, 0.093, 3]
[-0.318, -0.346, -0.659, 1.0, -1.0, -0.8, 7.2, 0.245, 0.294, 0.072, 3]
[-0.077, -0.165, -0.57, 0.907, -1.0, -0.744, 5.6, 0.197, 0.263, 0.085, 3]
[-0.052, -0.35, -0.665, 1.0, -1.0, -0.796, 6.5, 0.217, 0.321, 0.093, 4]
[-0.157, -0.29, -0.718, 0.989, -1.0, -0.791, 5.8, 0.173, 0.43, 0.066, 5]
[-0.032, -0.223, -0.434, 1.0, -0.999, -0.838, 0.0, 0.202, 0.446, 0.06, 0]
[-0.105, -0.104, -0.62, 1.0, -1.0, -0.722, 6.0, 0.217, 0.446, 0.068, 5]
[-0.242, -0.061, -0.614, 1.0, -1.0, -0.7, 0.0, 0.228, 0.438, 0.052, 5]
[-0.333, -0.279, -0.555, 1.0, -1.0, -0.718, 6.1, 0.207,

[-0.13, -0.147, -0.649, 0.989, -1.0, -0.762, 7.8, 0.234, 0.788, 0.019, 5]
[-0.315, -0.163, -0.599, 0.973, -1.0, -0.697, 4.7, 0.239, 0.818, 0.017, 5]
[-0.221, -0.019, -0.596, 0.988, -1.0, -0.717, 6.5, 0.192, 0.772, 0.05, 0]
[-0.084, -0.114, -0.552, 1.0, -1.0, -0.814, 6.3, 0.187, 0.788, 0.023, 3]
[-0.133, -0.189, -0.604, 0.994, -1.0, -0.672, 6.9, 0.217, 0.798, 0.02, 5]
[-0.207, -0.169, -0.601, 1.0, -1.0, -0.686, 5.1, 0.223, 0.793, 0.05, 5]
[-0.211, -0.286, -0.687, 1.0, -1.0, -0.788, 0.0, 0.239, 0.827, 0.047, 4]
[-0.221, -0.349, -0.716, 1.0, -1.0, -0.808, 7.5, 0.294, 0.182, 0.16, 5]
[-0.097, -0.06, -0.617, 0.987, -1.0, -0.74, 4.9, 0.314, 0.202, 0.133, 5]
[-0.22, -0.391, -0.642, 1.0, -1.0, -0.74, 6.8, 0.301, 0.223, 0.133, 3]
[-0.205, -0.224, -0.645, 1.0, -1.0, -0.696, 0.0, 0.307, 0.223, 0.107, 4]
[-0.244, -0.371, -0.738, 1.0, -1.0, -0.802, 6.4, 0.328, 0.207, 0.09, 5]
[-0.442, -0.559, -0.686, 1.0, -1.0, -0.804, 6.4, 0.314, 0.192, 0.078, 0]
[-0.016, -0.157, -0.552, 1.0, -1.0, -0.804, 6.1, 0.

[-0.083, -0.266, -0.593, 1.0, -1.0, -0.778, 0.0, 0.088, 0.803, 0.101, 1]
[-0.004, -0.329, -0.686, 0.999, -1.0, -0.77, 6.7, 0.126, 0.793, 0.083, 5]
[-0.031, -0.338, -0.63, 1.0, -1.0, -0.719, 6.5, 0.101, 0.793, 0.062, 4]
[-0.064, -0.14, -0.5, 0.976, -1.0, -0.628, 8.2, 0.137, 0.827, 0.058, 5]
[-0.312, -0.606, -0.636, 1.0, -1.0, -0.777, 6.5, 0.14, 0.844, 0.046, 1]
[-0.172, -0.152, -0.567, 0.993, -1.0, -0.655, 4.7, 0.113, 0.818, 0.028, 3]
[-0.149, 0.007, -0.609, 1.0, -1.0, -0.707, 6.4, 0.116, 0.822, 0.036, 0]
[-0.15, 0.007, -0.609, 1.0, -1.0, -0.707, 6.4, 0.119, 0.818, 0.028, 0]
[-0.081, -0.236, -0.564, 0.984, -1.0, -0.687, 5.9, 0.133, 0.798, 0.047, 5]
[-0.453, -0.339, -0.678, 1.0, -1.0, -0.75, 4.9, 0.133, 0.772, 0.07, 3]
[-0.271, -0.275, -0.665, 1.0, -1.0, -0.863, 6.1, 0.288, 0.212, 0.035, 5]
[-0.123, -0.455, -0.67, 0.999, -1.0, -0.793, 7.1, 0.294, 0.223, 0.05, 4]
[-0.226, -0.233, -0.539, 0.986, -1.0, -0.714, 5.1, 0.288, 0.251, 0.066, 5]
[-0.016, -0.157, -0.552, 1.0, -1.0, -0.804, 6.1, 0.2

[-0.143, -0.141, -0.673, 0.999, -1.0, -0.717, 2.7, 0.301, 0.239, 0.129, 4]
[-0.22, -0.391, -0.642, 1.0, -1.0, -0.74, 6.8, 0.301, 0.223, 0.133, 3]
[-0.244, -0.371, -0.738, 1.0, -1.0, -0.802, 6.4, 0.328, 0.207, 0.09, 5]
[-0.442, -0.559, -0.686, 1.0, -1.0, -0.804, 6.4, 0.314, 0.192, 0.078, 0]
[-0.016, -0.157, -0.552, 1.0, -1.0, -0.804, 6.1, 0.288, 0.212, 0.078, 3]
[-0.123, -0.455, -0.67, 0.999, -1.0, -0.793, 7.1, 0.294, 0.223, 0.05, 4]
[-0.124, -0.206, -0.564, 0.971, -1.0, -0.69, 6.4, 0.335, 0.223, 0.041, 0]
[-0.097, -0.06, -0.617, 0.987, -1.0, -0.74, 4.9, 0.314, 0.202, 0.133, 5]
[-0.22, -0.391, -0.642, 1.0, -1.0, -0.74, 6.8, 0.301, 0.223, 0.133, 3]
[-0.221, -0.349, -0.716, 1.0, -1.0, -0.808, 7.5, 0.294, 0.182, 0.16, 5]
[-0.311, 0.041, -0.58, 1.0, -1.0, -0.665, 5.1, 0.314, 0.16, 0.129, 3]
[-0.12, -0.255, -0.531, 0.97, -1.0, -0.759, 3.6, 0.307, 0.144, 0.11, 4]
[-0.082, -0.209, -0.484, 0.975, -1.0, -0.659, 6.0, 0.275, 0.187, 0.107, 5]
[-0.442, -0.559, -0.686, 1.0, -1.0, -0.804, 6.4, 0.314, 

...

[-0.137, -0.364, -0.703, 1.0, -1.0, -0.788, 4.8, 0.156, 0.547, 0.042, 5]
[-0.24, -0.296, -0.683, 0.994, -1.0, -0.758, 6.0, 0.152, 0.562, 0.047, 5]
[-0.388, -0.401, -0.733, 1.0, -1.0, -0.775, 8.4, 0.192, 0.578, 0.08, 4]
[-0.377, -0.303, -0.693, 1.0, -1.0, -0.749, 6.2, 0.178, 0.593, 0.047, 5]
[-0.295, -0.398, -0.717, 1.0, -1.0, -0.748, 6.3, 0.192, 0.615, 0.06, 1]
[0.151, -0.094, -0.584, 0.999, -1.0, -0.729, 5.7, 0.207, 0.37, 0.07, 2]
[-0.246, -0.299, -0.628, 1.0, -1.0, -0.754, 6.5, 0.217, 0.378, 0.066, 4]
[-0.062, -0.009, -0.48, 0.995, -1.0, -0.651, 3.8, 0.251, 0.378, 0.05, 5]
[-0.316, -0.348, -0.675, 1.0, -1.0, -0.814, 6.8, 0.202, 0.385, 0.09, 3]
[0.113, -0.104, -0.67, 0.974, -1.0, -0.85, 6.1, 0.239, 0.349, 0.072, 5]
[-0.256, -0.24, -0.734, 1.0, -1.0, -0.865, 6.6, 0.212, 0.335, 0.07, 1]
[-0.06, -0.358, -0.707, 1.0, -1.0, -0.823, 5.7, 0.182, 0.328, 0.098, 5]
[0.212, -0.382, -0.687, 0.993, -1.0, -0.792, 4.2, 0.173, 0.328, 0.113, 2]
[-0.169, -0.135, -0.676, 0.963, -1.0, -0.75, 5.4, 0.152, 

[-0.358, -0.262, -0.666, 1.0, -1.0, -0.72, 5.6, 0.187, 0.63, 0.045, 2]
[-0.295, -0.398, -0.717, 1.0, -1.0, -0.748, 6.3, 0.192, 0.615, 0.06, 1]
[-0.377, -0.303, -0.693, 1.0, -1.0, -0.749, 6.2, 0.178, 0.593, 0.047, 5]
[-0.388, -0.401, -0.733, 1.0, -1.0, -0.775, 8.4, 0.192, 0.578, 0.08, 4]
[-0.24, -0.296, -0.683, 0.994, -1.0, -0.758, 6.0, 0.152, 0.562, 0.047, 5]
[0.04, -0.377, -0.662, 1.0, -1.0, -0.799, 4.3, 0.173, 0.608, 0.024, 2]
[-0.293, -0.352, -0.644, 1.0, -1.0, -0.736, 6.5, 0.202, 0.562, 0.036, 4]
[-0.068, -0.142, -0.669, 1.0, -1.0, -0.76, 6.3, 0.239, 0.446, 0.093, 5]
[0.052, -0.244, -0.691, 0.991, -1.0, -0.841, 4.9, 0.275, 0.453, 0.06, 4]
[-0.105, -0.104, -0.62, 1.0, -1.0, -0.722, 6.0, 0.217, 0.446, 0.068, 5]
[-0.242, -0.061, -0.614, 1.0, -1.0, -0.7, 0.0, 0.228, 0.438, 0.052, 5]
[-0.333, -0.279, -0.555, 1.0, -1.0, -0.718, 6.1, 0.207, 0.415, 0.046, 4]
[-0.062, -0.076, -0.62, 0.986, -1.0, -0.774, 6.3, 0.187, 0.423, 0.08, 3]
[-0.157, -0.29, -0.718, 0.989, -1.0, -0.791, 5.8, 0.173, 0.4

[-0.253, -0.168, -0.567, 0.983, -1.0, -0.733, 6.2, 0.156, 0.686, 0.074, 5]
[-0.204, -0.182, -0.672, 1.0, -1.0, -0.766, 6.7, 0.14, 0.672, 0.083, 5]
[-0.073, -0.227, -0.626, 1.0, -1.0, -0.764, 5.6, 0.113, 0.658, 0.049, 3]
[-0.228, -0.292, -0.575, 0.994, -1.0, -0.69, 7.5, 0.169, 0.644, 0.062, 2]
[-0.019, -0.307, -0.626, 1.0, -1.0, -0.736, 7.0, 0.129, 0.453, 0.095, 3]
[-0.145, -0.26, -0.621, 1.0, -1.0, -0.72, 6.9, 0.16, 0.461, 0.058, 0]
[-0.34, -0.097, -0.594, 0.969, -1.0, -0.705, 5.0, 0.156, 0.423, 0.06, 4]
[-0.257, -0.302, -0.69, 1.0, -1.0, -0.748, 5.9, 0.16, 0.453, 0.033, 2]
[-0.062, -0.076, -0.62, 0.986, -1.0, -0.774, 6.3, 0.187, 0.423, 0.08, 3]
[-0.333, -0.279, -0.555, 1.0, -1.0, -0.718, 6.1, 0.207, 0.415, 0.046, 4]
[-0.305, -0.195, -0.672, 1.0, -1.0, -0.743, 0.0, 0.223, 0.453, 0.053, 5]
[-0.527, -0.201, -0.569, 1.0, -1.0, -0.62, 0.0, 0.234, 0.477, 0.031, 0]
[-0.242, -0.061, -0.614, 1.0, -1.0, -0.7, 0.0, 0.228, 0.438, 0.052, 5]
[-0.409, -0.515, -0.714, 1.0, -1.0, -0.784, 0.0, 0.202, 0

[-0.24, -0.296, -0.683, 0.994, -1.0, -0.758, 6.0, 0.152, 0.562, 0.047, 5]
[-0.388, -0.401, -0.733, 1.0, -1.0, -0.775, 8.4, 0.192, 0.578, 0.08, 4]
[-0.377, -0.303, -0.693, 1.0, -1.0, -0.749, 6.2, 0.178, 0.593, 0.047, 5]
[-0.295, -0.398, -0.717, 1.0, -1.0, -0.748, 6.3, 0.192, 0.615, 0.06, 1]
[-0.265, -0.276, -0.675, 1.0, -1.0, -0.756, 5.2, 0.245, 0.608, 0.046, 5]
[-0.295, -0.311, -0.702, 0.996, -1.0, -0.744, 5.7, 0.228, 0.637, 0.029, 5]
[-0.156, -0.088, -0.653, 0.989, -1.0, -0.712, 5.1, 0.251, 0.658, 0.046, 2]
[-0.233, -0.421, -0.652, 1.0, -1.0, -0.738, 7.3, 0.223, 0.658, 0.08, 2]
[-0.268, -0.174, -0.648, 0.995, -1.0, -0.737, 6.4, 0.245, 0.622, 0.09, 4]
[-0.141, -0.172, -0.649, 0.992, -1.0, -0.746, 4.8, 0.212, 0.615, 0.045, 1]
[-0.295, -0.398, -0.717, 1.0, -1.0, -0.748, 6.3, 0.192, 0.615, 0.06, 1]
[-0.377, -0.303, -0.693, 1.0, -1.0, -0.749, 6.2, 0.178, 0.593, 0.047, 5]
[-0.388, -0.401, -0.733, 1.0, -1.0, -0.775, 8.4, 0.192, 0.578, 0.08, 4]
[-0.293, -0.352, -0.644, 1.0, -1.0, -0.736, 6.5,

[-0.307, -0.021, -0.692, 0.967, -1.0, -0.741, 4.9, 0.148, 0.531, 0.064, 5]
[-0.195, -0.272, -0.56, 1.0, -1.0, -0.656, 4.5, 0.263, 0.43, 0.098, 5]
[-0.031, -0.04, -0.499, 1.0, -1.0, -0.569, 3.8, 0.251, 0.4, 0.101, 4]
[-0.237, -0.101, -0.736, 0.999, -1.0, -0.765, 5.7, 0.294, 0.4, 0.064, 4]
[0.007, 0.006, -0.462, 0.992, -1.0, -0.674, 4.2, 0.294, 0.423, 0.053, 5]
[-0.38, -0.085, -0.65, 0.993, -1.0, -0.707, 5.8, 0.321, 0.378, 0.032, 4]
[-0.386, -0.249, -0.661, 1.0, -1.0, -0.729, 6.0, 0.328, 0.392, 0.034, 1]
[-0.013, -0.119, -0.641, 0.978, -1.0, -0.79, 5.8, 0.342, 0.407, 0.058, 5]
[-0.28, -0.16, -0.732, 0.991, -1.0, -0.811, 6.9, 0.321, 0.37, 0.042, 2]
[0.077, -0.378, -0.72, 0.99, -1.0, -0.836, 5.3, 0.307, 0.349, 0.085, 1]
[-0.353, 0.038, -0.644, 0.999, -1.0, -0.738, 6.3, 0.294, 0.335, 0.064, 3]
[-0.058, -0.046, -0.603, 0.967, -1.0, -0.712, 6.6, 0.08, 0.608, 0.072, 5]
[-0.133, -0.338, -0.72, 0.996, -1.0, -0.836, 5.3, 0.107, 0.644, 0.036, 5]
[-0.073, -0.227, -0.626, 1.0, -1.0, -0.764, 5.6, 0.1

In [8]:
tokenized = create_numerate_array(movies_id)

In [9]:
print(np.array(scenes_array).shape)
print(np.array(tokenized).shape)

(7460, 11)
(7460,)


In [10]:
id_size = 1
id_embed_size = 11
vocab_size = 458
rnn_units = 512

In [41]:
class MyModel(tf.keras.Model):
    def __init__(self, vocab_size, id_size, id_embed_size, rnn_units):
        super().__init__(self)
        self.id_embedding = tf.keras.layers.Embedding(vocab_size, id_embed_size)
        self.normalization = tf.keras.layers.BatchNormalization()
        self.concat = tf.keras.layers.Concatenate()
        self.gru = tf.keras.layers.GRU(rnn_units,
                                       return_sequences=True,
                                       return_state=True)
        self.dense = tf.keras.layers.Dense(vocab_size, activation=activations.relu)

    def call(self, _input, states=None, return_state=False, training=False):
        ids = self.id_embedding(_input[0])
        features = self.normalization(_input[1])
        x = self.concat([ids, features])
        if states is None:
            states = self.gru.get_initial_state(x)
        x, states = self.gru(x, initial_state=states, training=training)
        x = self.dense(x)

        if return_state:
            return x, states
        else:
            return x
        
    def predict(self, x, history, batch_size=None, verbose=0, steps=None, callbacks=None, max_queue_size=10,
    workers=1, use_multiprocessing=False,):
        predictions = super().predict(x, batch_size, verbose, steps, callbacks, max_queue_size, workers, use_multiprocessing)
        predicted_values = []
        for i in range(len(predictions)):
            output = predictions[-1][0]
            mask = []
            for i in range(vocab_size):
                if i + 1 in history:
                    mask.append(1)
                else:
                    mask.append(0)
            masked_prediction = np.ma.array(output, mask=mask)
            predicted_values.append(np.argmax(masked_prediction) + 1)
            
        
        return predicted_values

In [42]:
train_tokenized = tokenized[:4500]
test_tokenized = tokenized[4500:7460]

train_scenes = scenes_array[:4500]
test_scenes = scenes_array[4500:7460]

ids_ds = tf.data.Dataset.from_tensor_slices(train_tokenized)
features_ds = tf.data.Dataset.from_tensor_slices(train_scenes)

ids_ds_test = tf.data.Dataset.from_tensor_slices(test_tokenized)
features_ds_test = tf.data.Dataset.from_tensor_slices(test_scenes)

ds_test = tf.data.Dataset.zip((ids_ds_test, features_ds_test))

ds = tf.data.Dataset.zip((ids_ds, features_ds))
seq_length = 10
examples_per_epoch = len(tokenized) // (seq_length + 1)

sequences = ds.batch(seq_length + 1, drop_remainder=True)
ids_seq = ids_ds.batch(seq_length + 1, drop_remainder=True)
features_seq = features_ds.batch(seq_length + 1, drop_remainder=True)
sequences_test = ds_test.batch(seq_length + 1, drop_remainder=True)

ds

<ZipDataset shapes: ((), (11,)), types: (tf.int32, tf.float64)>

In [43]:
def full_split_input_target(id_sequence, feature_sequence):
    input_text = (id_sequence[:-1], feature_sequence[:-1])
    target_text = (id_sequence[1:])
    return input_text, target_text

def split_input_target(seq):
    input_text = seq[:-1]
    target_text = seq[1:]
    return input_text, target_text

ds = sequences.map(full_split_input_target)
ids_ds = ids_seq.map(split_input_target)
features_ds = features_seq.map(split_input_target)
ds_test = sequences_test.map(full_split_input_target)

BATCH_SIZE = 32
TEST_BATCH_SIZE = 1
BUFFER_SIZE = 10000

ds = (
    ds
    .shuffle(BUFFER_SIZE)
    .batch(TEST_BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))


ids_ds = (
    ids_ds
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

features_ds = (
    features_ds
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE)
)

ds_test = (
    ds_test
    .shuffle(BUFFER_SIZE)
    .batch(TEST_BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

ds

<PrefetchDataset shapes: (((1, 10), (1, 10, 11)), (1, 10)), types: ((tf.int32, tf.float64), tf.int32)>

In [44]:
features_ds

<PrefetchDataset shapes: ((32, 10, 11), (32, 10, 11)), types: (tf.float64, tf.float64)>

In [45]:
model = MyModel(
    vocab_size=vocab_size,
    id_size=id_size,
    id_embed_size=id_embed_size,
    rnn_units=rnn_units)


In [46]:
for _input, _output in ds.take(1):
    example_batch_predictions = model(_input)
    print(example_batch_predictions, "# (batch_size, sequence_length, vocab_size)")
    sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
    sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()

    print(sampled_indices)

tf.Tensor(
[[[0.         0.         0.04718282 ... 0.         0.05762134 0.        ]
  [0.         0.         0.03285733 ... 0.         0.         0.        ]
  [0.         0.         0.07614313 ... 0.         0.07955685 0.        ]
  ...
  [0.         0.         0.03592366 ... 0.         0.         0.        ]
  [0.         0.         0.08389907 ... 0.         0.08249421 0.        ]
  [0.         0.         0.09974553 ... 0.         0.11680767 0.        ]]], shape=(1, 10, 458), dtype=float32) # (batch_size, sequence_length, vocab_size)
[236 271 235 294 167 229 173 134 147  67]


In [47]:
tf.config.run_functions_eagerly(True)
model.summary()

Model: "my_model_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      multiple                  5038      
_________________________________________________________________
batch_normalization_4 (Batch multiple                  44        
_________________________________________________________________
concatenate_4 (Concatenate)  multiple                  0         
_________________________________________________________________
gru_4 (GRU)                  multiple                  823296    
_________________________________________________________________
dense_4 (Dense)              multiple                  234954    
Total params: 1,063,332
Trainable params: 1,063,310
Non-trainable params: 22
_________________________________________________________________


In [48]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

In [49]:
model.compile(optimizer=tf.keras.optimizers.Adam(), loss=loss, metrics=[tf.keras.metrics.SparseCategoricalAccuracy(
    name='sparse_categorical_accuracy', dtype=None
)])

In [50]:
EPOCHS = 60
history = model.fit(ds, validation_data=ds_test, epochs=EPOCHS)

Epoch 1/60
409/409 [==============================] - 25s 60ms/step - loss: 6.0038 - sparse_categorical_accuracy: 0.0369 - val_loss: 5.6304 - val_sparse_categorical_accuracy: 0.0855
Epoch 2/60
409/409 [==============================] - 25s 60ms/step - loss: 5.3961 - sparse_categorical_accuracy: 0.1386 - val_loss: 4.9852 - val_sparse_categorical_accuracy: 0.2041
Epoch 3/60
409/409 [==============================] - 24s 58ms/step - loss: 4.6667 - sparse_categorical_accuracy: 0.2652 - val_loss: 4.7148 - val_sparse_categorical_accuracy: 0.2520
Epoch 4/60
409/409 [==============================] - 24s 59ms/step - loss: 4.2940 - sparse_categorical_accuracy: 0.3370 - val_loss: 4.4859 - val_sparse_categorical_accuracy: 0.2929
Epoch 5/60
409/409 [==============================] - 24s 60ms/step - loss: 4.0278 - sparse_categorical_accuracy: 0.3714 - val_loss: 4.4279 - val_sparse_categorical_accuracy: 0.3007
Epoch 6/60
409/409 [==============================] - 25s 61ms/step - loss: 3.8525 - spars

Epoch 46/60
409/409 [==============================] - 24s 58ms/step - loss: 3.6089 - sparse_categorical_accuracy: 0.4147 - val_loss: 4.5509 - val_sparse_categorical_accuracy: 0.3019
Epoch 47/60
409/409 [==============================] - 25s 61ms/step - loss: 3.5998 - sparse_categorical_accuracy: 0.4148 - val_loss: 4.5879 - val_sparse_categorical_accuracy: 0.2918
Epoch 48/60
409/409 [==============================] - 25s 61ms/step - loss: 3.6532 - sparse_categorical_accuracy: 0.4079 - val_loss: 4.5870 - val_sparse_categorical_accuracy: 0.2885
Epoch 49/60
409/409 [==============================] - 26s 64ms/step - loss: 3.6857 - sparse_categorical_accuracy: 0.4088 - val_loss: 4.4973 - val_sparse_categorical_accuracy: 0.2952
Epoch 50/60
409/409 [==============================] - 28s 70ms/step - loss: 3.6208 - sparse_categorical_accuracy: 0.4146 - val_loss: 4.4749 - val_sparse_categorical_accuracy: 0.3045
Epoch 51/60
409/409 [==============================] - 31s 75ms/step - loss: 3.5903 -

In [51]:
ds_test

<PrefetchDataset shapes: (((1, 10), (1, 10, 11)), (1, 10)), types: ((tf.int32, tf.float64), tf.int32)>

In [52]:
model.evaluate(ds_test, batch_size=TEST_BATCH_SIZE, return_dict=True)

269/269 [==============================] - 5s 18ms/step - loss: 4.4554 - sparse_categorical_accuracy: 0.3100


{'loss': 4.4553704261779785,
 'sparse_categorical_accuracy': 0.31003716588020325}

In [53]:
for _input, _output in ds_test.take(1):
    values = model.predict(_input, _input[0].numpy())
    print(values, _output.numpy()[0][-1])

[328] 408


In [54]:
print(vocab[_output.numpy()[0][-1] - 1])
print(vocab[values[0] - 1])

db37582459c34f8acdfaf38213a8f8
b2248b454f81373321d4cbfbd226d3


In [55]:
def getVideoInfoByName(name):
    info_csv = info_folder + '/' + name + '.csv'
    info_df =  pd.read_csv(info_csv, index_col=None, header=0)
    return getVideoInfo(info_df.iloc[: , 4:])

In [56]:
true_info = getVideoInfoByName(vocab[_output.numpy()[0][-1] - 1])
output_info = getVideoInfoByName(vocab[values[0] - 1])

TypeError: getVideoInfo() missing 1 required positional argument: 'trailers_row'

In [ ]:
true_info = np.nan_to_num(true_info, copy=False, nan=0.0, posinf=0.0, neginf=0.0)
output_info = np.nan_to_num(output_info, copy=False, nan=0.0, posinf=0.0, neginf=0.0)

In [ ]:
for i in range(id_embed_size):
    print(true_info[i])
    print(output_info[i])

In [ ]:
input_info = []
for i in range(10):
    print(i, getVideoInfoByName(vocab[_input[0].numpy()[0][i] - 1]))
    print("OUTPUT:", output_info)

In [ ]:
plt.plot(history.history['sparse_categorical_accuracy'])
plt.plot(history.history['val_sparse_categorical_accuracy'])
plt.title('Точность модели')
plt.ylabel('точность')
plt.xlabel('эпоха')
plt.legend(['тренировочные данные', 'тестовые данные'], loc='upper left')
plt.show()
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Потери модели')
plt.ylabel('потери')
plt.xlabel('эпоха')
plt.legend(['тренировочные данные', 'тестовые данные'], loc='upper left')
plt.show()